In [19]:
import mlflow
import scipy
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer

TRACKING_URL = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_URL)

In [31]:
model_uri = f"./models/models_mlflow"
booster = mlflow.xgboost.load_model(model_uri)

In [2]:
#loaded_model = mlflow.pyfunc.load_model('s3://mlflow-models-alexey/1/6dd459b11b4e48dc862f4e1019d166f6/artifacts/model/')

In [88]:

request = {
        'area_living': 120,
        'area_land': 1000,
        'n_rooms': 5,
        'year': 2014,
        'price': 240000
}

In [124]:
request = {
    'Unnamed: 0': {0: 134143},
    'date': {0: '2014-04-17'},
    'position_wgs': {0: 'POINT (-1.761870133979281 48.16737611723474)'},
    'x_lbt93': {0: 346197.91657676897},
    'y_lbt93': {0: 6795895.228610299},
    'category': {0: 'H'},
    'area_living': {0: 120.0},
    'area_land': {0: 609.0},
    'n_rooms': {0: 2},
    'shape_wgs': {0: 'MULTIPOLYGON (((-1.761755499999999 48.16739229999997, -1.7616759 48.16739359999997, -1.761699299999999 48.16723969999998, -1.761786499999999 48.16726199999997, -1.761841899999999 48.16727109999999, -1.761952399999999 48.16728159999998, -1.761974399999999 48.16728379999998, -1.762010799999998 48.16728869999996, -1.762036999999998 48.16729169999997, -1.76211 48.16730209999999, -1.762086699999999 48.16737089999999, -1.762092299999999 48.16737169999998, -1.762063699999999 48.16742789999997, -1.762042399999998 48.16748969999998, -1.7618452 48.16746759999998, -1.761848899999999 48.16745949999999, -1.761763399999999 48.16745069999997, -1.761739199999999 48.16744789999998, -1.761755499999999 48.16739229999997)))'},
    'price': {0: 17000.0},
    'year': {0: 2014}
}

In [89]:
import numpy as np
import scipy
import sklearn
from sklearn.feature_extraction import DictVectorizer

def add_features(
    df_train: pd.DataFrame, df_val: pd.DataFrame
) -> tuple(
    [
        scipy.sparse._csr.csr_matrix,
        scipy.sparse._csr.csr_matrix,
        np.ndarray,
        np.ndarray,
        sklearn.feature_extraction.DictVectorizer,
    ]
):
    features = ["area_living", "area_land", "n_rooms", "year", "price"]


    dv = DictVectorizer()

    train_dicts = df_train[features].to_dict(orient="records")
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[features].to_dict(orient="records")
    X_val = dv.transform(val_dicts)

    y_train = df_train["price"].values
    y_val = df_val["price"].values
    return X_train, X_val, y_train, y_val, dv

In [125]:
TRAIN_PATH = "../data/processed/train.csv"
SINGLE_PATH = "../data/processed/single.csv"
df_train = pd.read_csv(TRAIN_PATH)
#df_single = pd.read_csv(SINGLE_PATH)
dataframe = pd.DataFrame.from_dict(request)

X_train, X_pred, y_train, y_pred, dv = add_features(df_train, dataframe)
pred = xgb.DMatrix(X_pred, label=y_pred)

   Unnamed: 0        date                                  position_wgs  \
0      134143  2014-04-17  POINT (-1.761870133979281 48.16737611723474)   

         x_lbt93       y_lbt93 category  area_living  area_land  n_rooms  \
0  346197.916577  6.795895e+06        H        120.0      609.0        2   

                                           shape_wgs    price  year  
0  MULTIPOLYGON (((-1.761755499999999 48.16739229...  17000.0  2014  


In [126]:
booster.predict(pred)

array([15994.528], dtype=float32)

In [68]:
booster.feature_names


In [114]:
dataframe.to_dict()


{'Unnamed: 0': {0: 134143},
 'date': {0: '2014-04-17'},
 'position_wgs': {0: 'POINT (-1.761870133979281 48.16737611723474)'},
 'x_lbt93': {0: 346197.91657676897},
 'y_lbt93': {0: 6795895.228610299},
 'category': {0: 'H'},
 'area_living': {0: 68.0},
 'area_land': {0: 609.0},
 'n_rooms': {0: 2},
 'shape_wgs': {0: 'MULTIPOLYGON (((-1.761755499999999 48.16739229999997, -1.7616759 48.16739359999997, -1.761699299999999 48.16723969999998, -1.761786499999999 48.16726199999997, -1.761841899999999 48.16727109999999, -1.761952399999999 48.16728159999998, -1.761974399999999 48.16728379999998, -1.762010799999998 48.16728869999996, -1.762036999999998 48.16729169999997, -1.76211 48.16730209999999, -1.762086699999999 48.16737089999999, -1.762092299999999 48.16737169999998, -1.762063699999999 48.16742789999997, -1.762042399999998 48.16748969999998, -1.7618452 48.16746759999998, -1.761848899999999 48.16745949999999, -1.761763399999999 48.16745069999997, -1.761739199999999 48.16744789999998, -1.761755499